In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [5]:
# Description:
#   Exercise12 notebook.
#
# Copyright (C) 2018 Santiago Cortes, Juha Ylioinas
#
# This software is distributed under the GNU General Public 
# Licence (version 2 or later); please refer to the file 
# Licence.txt, included with the software, for details.

# Preparations
import numpy as np
import matplotlib.pyplot as plt

from utils import from_data_file, theta_to_model, model_to_theta, initial_model, logistic, \
    log_sum_exp_over_rows, classification_performance

# CS-E4850 Computer Vision Exercise Round 12

For these exercises you will need a python environment with necessary packages installed. For installation, see the README in the main page of this assignments repository.

The problems should be solved before the exercise session and solutions returned via the
MyCourses page. Upload two files: (1) a PDF file containing your written answers to
all problems, (2) a notebook containing the source code for the problem 3. Scanned
handwritten solutions are ok for problem 2. Notice also that the last two problems can
be done without solving problem 2.

## Exercise 1 - Course feedback (worth of 1 bonus point in similar manner as other tasks)
Fill in the official course feedback through the link that you should have received to your
email. The feedback collection is anonymous. The system separately reports the emails of
students, who have returned the feedback, but they are not associated with the answers.

__Additional material related to the following tasks is available on the MyCourses page in
Exercise12.zip, where the slides provide some background about the concepts covered
below. It may be helpful to check that material in case the problem context is not clear.__

## Exercise 2 - Neural  networks  and  backpropagation
This is a pen-&-paper problem, see Exercise2_pen&paper.pdf for the questions.

## Exercise 3 - Image  classification  using  a  neural  network
The exercise problem 2 above gives the solution to Part 2 of the second programming
assignment of professor Hinton’s course “Introduction to neural networks and machine
learning” held at the University of Toronto (https://www.cs.toronto.edu/~tijmen/csc321/ ). The assignment and related material are available at
https://www.cs.toronto.edu/~tijmen/csc321/assignments/a2/.

Check out the contents of the above web page and complete
the programming task of Part 2 according to the instructions. The solution for the
pen and paper part of the task is already given above. Hence, the programming part is
a relatively straightforward implementation and can be done without carrying out the
derivations since the results of the derivations are already given in __Exercise 2__ above.

In [2]:
def test_gradient(model, data, wd_coefficient):
    import sys
    base_theta = model_to_theta(model)
    h = 1e-2
    correctness_threshold = 1e-5
    analytic_gradient_struct = d_loss_by_d_model(model, data, wd_coefficient)

    analytic_gradient = model_to_theta(analytic_gradient_struct);
    if True in np.isnan(analytic_gradient) or True in np.isinf(analytic_gradient):
        sys.exit('Your gradient computation produced a NaN or infinity. That is an error.')
    # We want to test the gradient not for every element of theta, because that's a 
    # lot of work. Instead, we test for only a few elements. If there's an error, this 
    # is probably enough to find that error.
    # We want to first test the hid_to_class gradient, because that's most likely 
    # to be correct (it's the easier one).
    # Let's build a list of theta indices to check. We'll check 20 elements of 
    # hid_to_class, and 80 elements of input_to_hid (it's bigger than hid_to_class).
    input_to_hid_theta_size = model['input_to_hid'].size
    hid_to_class_theta_size = model['hid_to_class'].size
    big_prime = 1299721; # 1299721 is prime and thus ensures a somewhat random-like selection of indices.
    hid_to_class_indices_to_check = np.mod(big_prime * np.arange(20), hid_to_class_theta_size) \
                                        + input_to_hid_theta_size
    input_to_hid_indices_to_check = np.mod(big_prime * np.arange(80), input_to_hid_theta_size)
    a = hid_to_class_indices_to_check[np.newaxis,:]
    b = input_to_hid_indices_to_check[np.newaxis,:]
    indices_to_check = np.ravel(np.hstack((a,b)))

    for i in range(100):
        test_index = indices_to_check[i]
        analytic_here = analytic_gradient[test_index]
        theta_step = base_theta * 0
        theta_step[test_index] = h
        contribution_distances = np.array([-4.,  -3.,  -2.,  -1.,   1.,   2.,   3.,   4.])
        contribution_weights = np.array([1/280., -4/105., 1/5., -4/5., 4/5., -1/5., 4/105., -1/280.])
        temp = 0;
        for contribution_index in range(8):
            temp = temp + loss(theta_to_model(base_theta + theta_step * \
                                              contribution_distances[contribution_index]), data, wd_coefficient) * \
                                                contribution_weights[contribution_index]
        fd_here = temp / h
        diff = np.abs(analytic_here - fd_here)
        
        if True in (diff > correctness_threshold) and \
            True in (diff / (np.abs(analytic_here) + np.abs(fd_here)) > correctness_threshold):
            part_names = ['input_to_hid', 'hid_to_class']
            sys.exit('Theta element #{} (part of {}), with value {}, has finite difference gradient {} but analytic gradient {}. That looks like an error.\n'.format(test_index, part_names[(i<=20)], base_theta[test_index], fd_here, analytic_here))
        if i==20: 
            print('Gradient test passed for hid_to_class. ')
        if i==100: 
            print('Gradient test passed for input_to_hid. ')
    print('Gradient test passed. That means that the gradient that your code computed is within 0.001%% of the gradient that the finite difference approximation computed, so the gradient calculation procedure is probably correct (not certainly, but probably).\n')
    
def forward_pass(model, data):
    # This function does the forward pass through the network: calculating the states of all units, and some related data. 
    # This function is used in functions loss() and d_loss_by_d_model().  
  
    # model.input_to_hid is a matrix of size <number of hidden units> by <number of inputs i.e. 256>. It contains the weights from the input units to the hidden units.
    # model.hid_to_class is a matrix of size <number of classes i.e. 10> by <number of hidden units>. It contains the weights from the hidden units to the softmax units.
    # data.inputs is a matrix of size <number of inputs i.e. 256> by <number of data cases>. Each column describes a different data case. 
    # data.targets is a matrix of size <number of classes i.e. 10> by <number of data cases>. Each column describes a different data case. It contains a one-of-N encoding of the class, i.e. one element in every column is 1 and the others are 0.
    
    hid_input = np.dot(model['input_to_hid'], data['inputs']) # input to the hidden units, i.e. before the logistic. size: <number of hidden units> by <number of data cases>
    hid_output = logistic(hid_input) # output of the hidden units, i.e. after the logistic. size: <number of hidden units> by <number of data cases>
    class_input = np.dot(model['hid_to_class'], hid_output) # input to the components of the softmax. size: <number of classes, i.e. 10> by <number of data cases>
  
    # The following three lines of code implement the softmax.
    # However, it's written differently from what the lectures say.
    # In the lectures, a softmax is described using an exponential divided by a sum of exponentials.
    # What we do here is exactly equivalent (you can check the math or just check it in practice), but this is more numerically stable. 
    # "Numerically stable" means that this way, there will never be really big numbers involved.
    # The exponential in the lectures can lead to really big numbers, which are fine in mathematical equations, but can lead to all sorts of problems in Matlab
    # Matlab isn't well prepared to deal with really large numbers, like the number 10 to the power 1000. Computations with such numbers get unstable, so we avoid them.

    class_normalizer = log_sum_exp_over_rows(class_input) # log(sum(exp of class_input)) is what we subtract to get properly normalized log class probabilities. size: <1> by <number of data cases>
    log_class_prob = class_input - np.tile(class_normalizer, (class_input.shape[0], 1)) # log of probability of each class. size: <number of classes, i.e. 10> by <number of data cases>
    class_prob = np.exp(log_class_prob) # probability of each class. Each column (i.e. each case) sums to 1. size: <number of classes, i.e. 10> by <number of data cases>

    return hid_input, hid_output, class_input, log_class_prob, class_prob

def loss(model, data, wd_coefficient):
    hid_input, hid_output, class_input, log_class_prob, class_prob = forward_pass(model, data);
    classification_loss = -np.mean(np.sum(np.multiply(log_class_prob, data['target']), 0)) # select the right log class probability using that sum; then take the mean over all data cases.
    wd_loss = (np.sum(np.ravel(model['input_to_hid']) ** 2 ) + np.sum(np.ravel(model['hid_to_class']) ** 2 )) / 2. * wd_coefficient; # weight decay loss. very straightforward: E = 1/2 * wd_coeffecient * parameters^2
    ret = classification_loss + wd_loss
    return ret  

In [6]:
def d_loss_by_d_model(model, data, wd_coefficient):
    # model.input_to_hid is a matrix of size <number of hidden units> by <number of inputs i.e. 256>
    # model.hid_to_class is a matrix of size <number of classes i.e. 10> by <number of hidden units>
    # data.inputs is a matrix of size <number of inputs i.e. 256> by <number of data cases>
    # data.targets is a matrix of size <number of classes i.e. 10> by <number of data cases>

    # The returned object <ret> is supposed to be exactly like parameter <model>, i.e. it has fields ret.input_to_hid and ret.hid_to_class, and those are of the same shape as they are in <model>.
    # However, in <ret>, the contents of those matrices are gradients (d loss by d weight), instead of weights.
    ret = dict()
    # This is the only function that you're expected to change. Right now, it just returns a lot of zeros, which is obviously not the correct output. Your job is to change that.
    #--your-code-starts-here--#
    ret['input_to_hid'] = model['input_to_hid'] * 0;
    ret['hid_to_class'] = model['hid_to_class'] * 0;
    #--your-code-ends-here--#
    return ret

In [7]:
def a2(wd_coefficient, n_hid, n_iters, learning_rate, momentum_multiplier, do_early_stopping, mini_batch_size):
    model = initial_model(n_hid)
    datas = from_data_file()

    n_training_cases = datas['train']['inputs'].shape[1]
    if n_iters != 0:
        print("Now testing the gradient on the whole training set...")
        test_gradient(model, datas['train'], wd_coefficient)
    
    # optimization
    training_batch = dict()
    best_so_far = dict()
    theta = model_to_theta(model)
    momentum_speed = theta * 0.
    training_data_losses = []
    validation_data_losses = []
    if do_early_stopping:
        best_so_far['theta'] = -1 # this will be overwritten soon
        best_so_far['validation_loss'] = np.Inf
        best_so_far['after_n_iters'] = -1
        
    for optimization_iteration_i in range(1, n_iters+1):
        model = theta_to_model(theta)
        training_batch_start = np.mod((optimization_iteration_i-1) * mini_batch_size, n_training_cases);  
        training_batch['inputs'] = datas['train']['inputs'][:, training_batch_start : training_batch_start + mini_batch_size]
        training_batch['target'] = datas['train']['target'][:, training_batch_start : training_batch_start + mini_batch_size]
        gradient = model_to_theta(d_loss_by_d_model(model, training_batch, wd_coefficient))
        momentum_speed = np.multiply(momentum_speed, momentum_multiplier) - gradient;
        theta = theta + momentum_speed * learning_rate;
        model = theta_to_model(theta);
        training_data_losses.append(loss(model, datas['train'], wd_coefficient))
        validation_data_losses.append(loss(model, datas['val'], wd_coefficient))
        
        if do_early_stopping and validation_data_losses[-1] < best_so_far['validation_loss']:
            best_so_far['theta'] = theta; # this will be overwritten soon
            best_so_far['validation_loss'] = validation_data_losses[-1]
            best_so_far['after_n_iters'] = optimization_iteration_i
            
        if np.mod(optimization_iteration_i, np.round(n_iters / 10.)) == 0:
            print('After {} optimization iterations, training data loss is {}, and validation data loss is {}\n'.format(optimization_iteration_i, training_data_losses[-1], validation_data_losses[-1]))
    
        if optimization_iteration_i == n_iters: # check gradient again, this time with more typical parameters and with a different data size
            print('Now testing the gradient on just a mini-batch instead of the whole training set... ')
            test_gradient(model, training_batch, wd_coefficient)
            
    if do_early_stopping:
        print('Early stopping: validation loss was lowest after {} iterations. We chose the model that we had then.\n'.format(best_so_far['after_n_iters']))
        theta = best_so_far['theta']
    
    # the optimization is finished. Now do some reporting.
    model = theta_to_model(theta)
    if n_iters != 0:
        ax = plt.figure(1, figsize=(15,10))
        plt.plot(training_data_losses, 'b')
        plt.plot(validation_data_losses, 'r')
        plt.tick_params(labelsize=25)
        ax.legend(('training', 'validation'), fontsize=25)
        plt.ylabel('loss', fontsize=25);
        plt.xlabel('iteration number', fontsize=25);
        plt.show()
    
    datas2 = [datas['train'], datas['val'], datas['test']]
    data_names = ['training', 'validation', 'test'];
    for data_i in range(3):
        data = datas2[data_i]
        data_name = data_names[data_i]
        print('\nThe total loss on the {} data is {}\n'.format(data_name, loss(model, data, wd_coefficient)))
        print('The classification loss (i.e. without weight decay) on the {} data is {}\n'.format(data_name, loss(model, data, 0)));
        print('The classification error rate on the {} data is {}\n'.format(data_name, classification_performance(model, data)))

    

In [20]:
# Start training the neural network
#a2(0, 10, 70, 20.0, 0, False, 4) 


## Exercise 4 - Optimisation using backpropagation
Do Part 3 of the assignment which is available at
http://www.cs.toronto.edu/~tijmen/csc321/assignments/a2/

The task is to experiment with the given example code and report your findings.
There is no need to program anything in this part but completing it requires that Part 2
is successfully solved.

In [21]:
#try with learning_rate = 0.002, 0.01, 0.05, 0.2, 1.0, 5.0, and 20.0, and with and without momentum_multiplier=0.9
